# Load parameteres with traces from CSV and calculate their features

In [1]:
import os
import glob
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
os.chdir('../')

In [56]:
# from utils import calc_features_step, calc_ramp_sum_stats, find_spikes
from utils import calc_ramp_sum_stats, find_spikes

In [2]:
from dap import DAPcython
from dap.utils import obs_params, load_current
from tqdm import tqdm
from scipy.spatial import distance

In [ ]:
from utils import

In [3]:
dt = 1e-2
params, labels = obs_params(reduced_model=False)
data_dir = '/home/alteska/Desktop/LFI_DAP/data/rawData/2015_08_26b.dat'

In [4]:
# load the file
directory = './parameters/'
dir = glob.glob(directory + '*')

In [5]:
fname_start = dir[0].find('dap_')
fname_stop = dir[0].find('n_')
fname = dir[0][fname_start:fname_stop] + '.csv'

In [6]:
df_param = pd.read_csv(fname)
df_param.head()

,Unnamed: 0,gbar_nap,gbar_leak,gbar_nat,gbar_kdr
0,1x14,17.173732,1.074666,131.422942,3.410514
1,1x35,17.097219,1.039869,117.684230,3.737484
2,1x9,19.007380,0.991425,95.365731,2.976070
3,1x19,16.370543,1.028504,141.774573,3.430032
4,1x24,15.153335,1.100184,163.721094,3.055541


In [7]:
df_param.set_index('Unnamed: 0', inplace=True)
df_param.head()

,gbar_nap,gbar_leak,gbar_nat,gbar_kdr
Unnamed: 0,,,,
1x14,17.173732,1.074666,131.422942,3.410514
1x35,17.097219,1.039869,117.684230,3.737484
1x9,19.007380,0.991425,95.365731,2.976070
1x19,16.370543,1.028504,141.774573,3.430032
1x24,15.153335,1.100184,163.721094,3.055541


In [8]:
# calculate DAP
# load the input data
Ir, vr, tr, t_onr, t_offr, dtr = load_current(data_dir, protocol='rampIV', ramp_amp=3.1)
Is, vs, ts, t_ons, t_offs, dts = load_current(data_dir, protocol='IV', ramp_amp=1)

In [9]:
# define a model
dap = DAPcython(-75, params)

In [10]:
# run models on original parameters
U_step = dap.simulate(dts, ts, Is)
U_ramp = dap.simulate(dtr, tr, Ir)

In [11]:
# calculate the similarities
d_step = distance.euclidean(vs, U_step)
d_ramp = distance.euclidean(vr, U_ramp)

In [13]:
# run for all cells and save into the the DF
df_paramT = df_param.transpose()
df_paramT.head()

Unnamed: 0,1x14,1x35,1x9,1x19,1x24,1x16,1x31k,1x12,1x27,1x23,...,1x29,1x32,1x38,1x25,1x22,1x33,1x11k,1x21k,1x34,1x17
gbar_nap,17.173732,17.097219,19.007380,16.370543,15.153335,16.515891,14.387062,16.183388,16.586714,18.934190,...,17.687560,16.339341,15.024635,16.749955,15.366489,18.050893,19.543175,16.077841,16.412064,16.126095
gbar_leak,1.074666,1.039869,0.991425,1.028504,1.100184,1.121755,1.093874,1.035868,1.046909,1.025682,...,1.071797,0.990389,1.095737,1.124608,1.031578,1.071510,1.085424,0.998721,1.044613,1.050323
gbar_nat,131.422942,117.684230,95.365731,141.774573,163.721094,120.139337,164.149919,108.916781,169.625743,115.465286,...,95.583603,153.130788,178.050132,118.796892,166.834822,113.726997,125.532725,141.340125,160.322308,115.380782
gbar_kdr,3.410514,3.737484,2.976070,3.430032,3.055541,3.414985,3.128166,2.994278,3.174331,3.194553,...,3.464703,3.134960,3.207311,3.086094,3.103938,3.787249,3.164570,3.846042,3.103724,3.335470


In [20]:
U_steps, U_ramps = [], []

In [33]:
for i, j in tqdm(df_param.iterrows()):
    # get parameters
    par_temp = j.values

    # define a model
    dap = DAPcython(-75, j)

    # run model
    U_step_x = dap.simulate(dts, ts, Is)
    U_ramp_x = dap.simulate(dtr, tr, Ir)

    # run model
    U_steps.append(U_step_x)
    U_ramps.append(U_ramp_x)

    # calculate distance for both currents
    dis_step = distance.euclidean(vs, U_step_x)
    dis_ramp = distance.euclidean(vr, U_ramp_x)

    # save into new columns
    df_paramT.loc[i, 'distance_ramp'] = dis_ramp
    df_paramT.loc[i, 'distance_step'] = dis_step
    df_paramT.loc[i, 'distance_sum'] = dis_ramp + dis_step

34it [00:39,  1.14s/it]


In [35]:
U_steps[0]

In [38]:
stats = calc_features_step(U_steps[0], ts, dts, t_ons, t_offs)
stats

array([-69.09264538775403, 0.03299505873564305, 41,
       array([252.25, 252.35, 262.15, 274.15, 286.6 , 299.15, 311.75, 324.4 ,
       336.95, 349.55, 362.25, 374.95, 387.6 , 400.2 , 412.85, 425.5 ,
       438.15, 450.85, 463.5 , 476.15, 488.8 , 501.5 , 514.2 , 526.9 ,
       539.6 , 552.3 , 564.95, 577.7 , 590.35, 603.  , 615.65, 628.3 ,
       640.9 , 653.6 , 666.2 , 678.9 , 691.55, 704.2 , 716.9 , 729.55,
       742.25]),
       82.0, 12.250000000000004, 1.9984994370777323], dtype=object)

In [51]:
print(U_steps[15])

[-75.01744904 -74.90108798 -74.79126475 ... -52.22251678 -52.22002466
 -52.2388784 ]


In [57]:
def calc_features_step(U, t, dt, t_on, t_off):
    v = U.copy()
    N = v.shape[0]

    # resting potential
    rest_pot = np.mean(v[t<t_on])
    rest_pot_std = np.std(v[int(.9*t_on/dt):int(t_on/dt)])  
 
    ind = find_spikes(v)

    spike_times = np.array(t)[ind]
    spike_times_stim = spike_times[(spike_times > t_on)  & (spike_times < t_off)]
    ind_stim1 = ind[(spike_times > t_on) & (spike_times < t_off)]
    ind_stim = ind_stim1.astype(int)

    firing_rate = 1e3*np.absolute(spike_times_stim.shape[0]/(t_off-t_on))
    

    print(spike_times_stim)
    time_1st_spike = spike_times_stim[spike_times_stim>t_on][0]
    ISI = np.diff(spike_times_stim).astype(float)

    
    sum_stats_vec = np.array([
                rest_pot,
                rest_pot_std,
                len(spike_times_stim),
                spike_times_stim,
                firing_rate,
                ISI.mean(),
                ISI.std()
                ])


    return sum_stats_vec

In [58]:
stats_step = []


for i,u in enumerate(U_steps):
    print(i)
    stats = calc_features_step(u, ts, dts, t_ons, t_offs)
#     stats_step.append(stats)

0
[252.25 252.35 262.15 274.15 286.6  299.15 311.75 324.4  336.95 349.55
 362.25 374.95 387.6  400.2  412.85 425.5  438.15 450.85 463.5  476.15
 488.8  501.5  514.2  526.9  539.6  552.3  564.95 577.7  590.35 603.
 615.65 628.3  640.9  653.6  666.2  678.9  691.55 704.2  716.9  729.55
 742.25]
1
[253.75]
2
[254.6  261.6  268.55 275.45 282.35 289.25 296.15 303.05 309.95 316.85
 323.8  330.65 337.55 344.45 351.4  358.25 365.15 372.05 378.95 385.85
 392.75 399.65 406.55 413.45 420.35 427.25 434.15 441.05 447.95 454.85
 461.75 468.6  475.5  482.4  489.3  496.2  503.1  510.   516.9  523.8
 530.7  537.6  544.5  551.4  558.3  565.2  572.1  579.   585.9  592.8
 599.7  606.6  613.5  620.4  627.3  634.2  641.1  648.   654.9  661.8
 668.7  675.6  682.5  689.4  696.3  703.2  710.1  717.   723.9  730.8
 737.7  744.6 ]
3
[253.15 253.25 263.75 278.65 294.4  294.5  310.15 326.   341.8  357.6
 373.5  389.45 389.55 405.3  421.25 437.2  437.3  453.05 469.   469.1
 484.85 500.8  516.75 516.85 532.55 548.5  

/home/alteska/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:29: RuntimeWarning: Mean of empty slice.


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
# save the new data DataFrame
# df_paramT.to_csv(fname[:-4] + '_similarity.csv')

# remove the temp directory
# shutil.rmtree(directory)